In [1]:
import os
from sklearn.model_selection import KFold
import numpy as np

class CFG:
    class data:
        batch_size=32
        validation_size = 0.2
        lr = 1e-4
        epochs = 10  
        epsilon = 1e-8
        MAX_LEN = 128 #max sentence length
        seed_val = 42 #random seed
        k_folds = 10
        hidden_size = 768 #hidden layer size (embedding size) for feedforward net
#         PATH = "/content/drive/MyDrive/Minor Project1/"
        PATH = "./"

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1660 Ti


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import pandas as pd

df = pd.read_csv('ETHOS_binary.csv')

print('Number of training sentences: {:,}\n'.format(df.shape[0]))

sentences = df.text.values
labels = df.label.values

df.head(10)

Number of training sentences: 998



,text,label
0,You should know women's sports are a joke,1
1,You look like Sloth with deeper Down’s syndrome,1
2,You look like Russian and speak like Indian. B...,1
3,"Women deserve to be abused, I guess.",1
4,Women are made for making babies and cooking d...,1
5,why haven't you faggots taken the bi-pill yet,1
6,Why don’t you go beck your god to murder you? ...,1
7,why does that african american monkey hold gun...,1
8,Why does Bulgaria exist as a country? It's one...,1
9,Why do negroes tear down their own neighborhoo...,1


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#Load the Bert model.
print('Loading BERT Model...')
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

filepath = CFG.data.PATH + 'original.pth'
if not os.path.exists(filepath):
    torch.save(model.state_dict(), filepath) 
    
# Tell pytorch to run this model on the GPU.
model.cuda()

Loading BERT tokenizer...
Loading BERT Model...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                        # This function also supports truncation and conversion
                        # to pytorch tensors, but we need to do padding, so we
                        # can't use these features :( .
                        #max_length = 128,          # Truncate all sentences.
                        #return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (889 > 512). Running this sequence through the model will result in indexing errors


Original:  You should know women's sports are a joke
Token IDs: [101, 2017, 2323, 2113, 2308, 1005, 1055, 2998, 2024, 1037, 8257, 102]


In [7]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  889


In [8]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras_preprocessing.sequence import pad_sequences

print('\nPadding/truncating all sentences to %d values...' % CFG.data.MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=CFG.data.MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 128 values...

Padding token: "[PAD]", ID: 0

Done.


In [9]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [10]:
#one-hot encode labels
def one_hot(categorical_labels):
    num_categories = 2
    labels = np.zeros((categorical_labels.shape[0], num_categories))
    labels[:,0] = (categorical_labels == 0).astype(int)
    labels[:,1] = (categorical_labels == 1).astype(int)
    return labels

labels = one_hot(labels)

In [11]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup

# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
# train_inputs = torch.tensor(train_inputs)
# validation_inputs = torch.tensor(validation_inputs)

# train_labels = torch.tensor(train_labels)
# validation_labels = torch.tensor(validation_labels)

# train_masks = torch.tensor(train_masks)
# validation_masks = torch.tensor(validation_masks)

input_ids = torch.tensor(input_ids)
labels = torch.tensor(labels)
attention_masks = torch.tensor(attention_masks)

dataset = TensorDataset(input_ids, labels, attention_masks)

In [12]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = CFG.data.epochs

In [13]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
# def flat_accuracy(preds, labels):
#     labels_flat = labels.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_accuracy(preds, labels):
    y_hat = np.argmax(preds, axis=1).flatten()
    y = labels[:,1].flatten()
      
    #find tp, tn, fp, fn
    tp = ((y==1) & (y_hat==1)).sum()
    tn = ((y==0) & (y_hat==0)).sum()
    fp = ((y==0) & (y_hat==1)).sum()
    fn = ((y==1) & (y_hat==0)).sum()
    total = len(labels)
    return tp, tn, fp, fn, total
    
def get_accuracy(tp, tn, fp, fn, total):
    epsilon = CFG.data.epsilon

    def accuracy():
      return (tp + tn)/(total + epsilon) 

    def precision():
      return tp/(tp + fp + epsilon)
    
    def recall():
      return tp/(tp + fn + epsilon)

    def f1():
      p = precision()
      r = recall()
      return 2*p*r/(p + r + epsilon)

    return {
        'accuracy': accuracy(),
        'precision': precision(),
        'recall': recall(),
        'f1-score': f1()
    }

In [14]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for layer in range(12):
    model.bert.encoder.layer[layer].output.LayerNorm.register_forward_hook(get_activation(layer))

In [15]:
input_ids_ = input_ids[:5].to(device)
input_mask_ = attention_masks[:5].to(device)
input_labels_ = labels[:5].to(device)

In [16]:
input_ids_.shape

torch.Size([5, 128])

In [17]:
outputs = model(input_ids_, 
                token_type_ids=None, 
                attention_mask=input_mask_)

In [18]:
import copy
a1 = copy.deepcopy(activation)

In [19]:
print(a1[11].shape)
a1[11].mean(dim=1).shape
a1[11][:,0,:].shape

torch.Size([5, 128, 768])


torch.Size([5, 768])

In [20]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(CFG.data.hidden_size, CFG.data.hidden_size)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.leakyrelu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p = 0.2)
        self.fc1 = nn.Linear(CFG.data.hidden_size, 2) #input_size, output_size
        # self.fc2 = nn.Linear(64,64)
        # self.fc3 = nn.Linear(64,64)
        # self.fc4 = nn.Linear(64,10) #10 because 10 classes {0,1,2,3,4,5,6,7,8,9}
        
    def forward(self, x):
        x = self.dense(x)
        x = self.tanh(x)
        # x = self.relu(x)
#         x = self.leakyrelu(x)
        x = self.dropout(x)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.fc1(x)
        
        return x


In [21]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

for layer in range(12):
    model.bert.encoder.layer[layer].output.LayerNorm.register_forward_hook(get_activation(layer))

def prepare_data(inputs, masks, labels):
    inputs = inputs.to(device)
    masks = masks.to(device)
    labels = labels.to(device)
    outputs = model(inputs, 
                token_type_ids=None, 
                attention_mask=masks)
    # inputs = activation[11][:,0,:]
    inputs = activation[11].mean(dim=1) #exp1: taking mean of all tokens in last layer
    
    #exp2
#     cnt = 1e-8
#     inputs = torch.zeros_like(activation[0][:,0,:])
#     for i in range(9,12):
#       cnt += 1
#       inputs += activation[i][:,0,:]
#     inputs/=cnt
    
    return [inputs, labels]

In [22]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [23]:
batch_size = CFG.data.batch_size

def train1(train_data, validation_data, fold_id, exp_no):
    '''
      Data: (list) -> [inputs, masks, labels]
    '''
    
    net = Net()
    net.cuda()

    # Create the DataLoader for our training set.
    train_data = TensorDataset(*train_data)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create the DataLoader for our validation set.
    validation_data = TensorDataset(*validation_data)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

    # Function to validate model
    def validate_model(validation_dataloader):
        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        net.eval()

        # Tracking variables 
        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        tp, tn, fp, fn, total = 0, 0, 0, 0, 0
        # Evaluate data for one epoch
        for batch in validation_dataloader:
            
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            
            # Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = batch
            
            # Telling the model not to compute or store gradients, saving memory and
            # speeding up validation
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                b_inputs, b_labels = prepare_data(b_input_ids, b_input_mask, b_labels)

                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                logits = net(b_inputs)


            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            
            # Calculate the accuracy for this batch of test sentences.
            tp_, tn_, fp_, fn_, tot_ = flat_accuracy(logits, label_ids)
            
            tp+=tp_
            tn+=tn_
            fp+=fp_
            fn+=fn_
            total+=tot_

            # Track the number of batches
            nb_eval_steps += 1

        # Report the final accuracy for this validation run.
        acc_metrics = get_accuracy(tp, tn, fp, fn, total)
        for k, v in acc_metrics.items():
          print("{} : {:.5f},".format(k,v), end=" ")

        print("\nValidation took: {:}".format(format_time(time.time() - t0)))
        return acc_metrics
    

    best_metrics = {}
    best_f1_so_far = 0.0
    
    # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
    optimizer = AdamW(net.parameters(),
                      lr = CFG.data.lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                      eps = CFG.data.epsilon
                    )
    
    #define loss function
    criterion = nn.CrossEntropyLoss()

    # Total number of training steps is number of batches * number of epochs.
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    
    print('Current Best f1-score: {:.5}'.format(best_f1_so_far))

    # For each epoch...
    for epoch_i in range(0, epochs):
        
        # ========================================
        #               Training
        # ========================================
        
        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        
        # For each batch of training data...
        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_loss = 1

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        net.train()

        print('Training...')
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
                
                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            b_inputs, b_labels = prepare_data(b_input_ids, b_input_mask, b_labels)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            net.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = net(b_inputs)
            
            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple.
            loss = criterion(outputs, b_labels)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)            
        
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        accuracy_metrics = validate_model(validation_dataloader)
        current_f1 = accuracy_metrics['f1-score']
        if current_f1>best_f1_so_far:
            print('f1 score improved, old f1 = {:.5}, new f1 = {:.5}'.format(best_f1_so_far, current_f1))
            best_f1_so_far = current_f1
            best_metrics = accuracy_metrics
            print('Saving new weights...')
            torch.save(net.state_dict(), filepath + f'net_exp{exp_no}_{fold_id}.pth')

        elif current_f1 == best_f1_so_far:
            print('f1 score did not change')
            b_acc = 0.0
            if 'accuracy' in best_metrics:
                b_acc = best_metrics['accuracy']
            cur_acc = accuracy_metrics['accuracy']
            if cur_acc>b_acc:
                print('accuracy score improved, old f1 = {:.5}, new f1 = {:.5}'.format(b_acc, cur_acc))
                best_metrics = accuracy_metrics
                print('Saving new weights...')
                torch.save(net.state_dict(), filepath + f'net_exp{exp_no}_{fold_id}.pth')

    return best_metrics

In [24]:
kfold = KFold(n_splits=CFG.data.k_folds, shuffle=True)
best_f1_scores = {}
model.train(False)

for fold_id, (train_ids, val_ids) in enumerate(kfold.split(dataset)):

    print('------------------ Fold {} ------------------'.format(fold_id))

    train_inputs, train_masks, train_labels = input_ids[train_ids], attention_masks[train_ids], labels[train_ids]
    validation_inputs, validation_masks, validation_labels = input_ids[val_ids], attention_masks[val_ids], labels[val_ids]
    train_data = [train_inputs, train_masks, train_labels]
    validation_data = [validation_inputs, validation_masks, validation_labels]

    best_f1_scores[fold_id] = train1(train_data, validation_data, fold_id, 1)

print("")
print("Training complete!")

------------------ Fold 0 ------------------
Current Best f1-score: 0.0

======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.67
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.65000, precision : 0.83333, recall : 0.31915, f1-score : 0.46154, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.0, new f1 = 0.46154
Saving new weights...

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.59
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.70000, precision : 0.81481, recall : 0.46809, f1-score : 0.59459, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.46154, new f1 = 0.59459
Saving new weights...

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.57
  Training epoch took: 0:00:11

Running Validation...
accuracy : 0.69000, precision : 0.73529, recall : 0.53191, f1-score : 0.61728, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.59459, new f1 = 0.61728
Saving new weights...

=


  Average training loss: 0.68
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.70000, precision : 0.57143, recall : 0.47059, f1-score : 0.51613, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.0, new f1 = 0.51613
Saving new weights...

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.60
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.69000, precision : 0.54286, recall : 0.55882, f1-score : 0.55072, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.51613, new f1 = 0.55072
Saving new weights...

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.56
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.69000, precision : 0.53333, recall : 0.70588, f1-score : 0.60759, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.55072, new f1 = 0.60759
Saving new weights...

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.54
  Training epoch took: 0:00:11

Running Val


  Average training loss: 0.50
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.77000, precision : 0.79070, recall : 0.70833, f1-score : 0.74725, 
Validation took: 0:00:01
f1 score did not change

======== Epoch 10 / 10 ========
Training...

  Average training loss: 0.49
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.77000, precision : 0.79070, recall : 0.70833, f1-score : 0.74725, 
Validation took: 0:00:01
f1 score did not change
------------------ Fold 6 ------------------
Current Best f1-score: 0.0

======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.67
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.73000, precision : 0.84211, recall : 0.40000, f1-score : 0.54237, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.0, new f1 = 0.54237
Saving new weights...

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.59
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.77000, 


  Average training loss: 0.49
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.75758, precision : 0.71053, recall : 0.67500, f1-score : 0.69231, 
Validation took: 0:00:01
f1 score did not change

======== Epoch 10 / 10 ========
Training...

  Average training loss: 0.49
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.75758, precision : 0.72222, recall : 0.65000, f1-score : 0.68421, 
Validation took: 0:00:01
------------------ Fold 9 ------------------
Current Best f1-score: 0.0

======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.67
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.69697, precision : 0.66667, recall : 0.63636, f1-score : 0.65116, 
Validation took: 0:00:01
f1 score improved, old f1 = 0.0, new f1 = 0.65116
Saving new weights...

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.59
  Training epoch took: 0:00:10

Running Validation...
accuracy : 0.71717, precision : 0.69048, rec

In [25]:
for fold_id in range(10):
    print(f"{fold_id}: ", end = " ")
    for k, v in best_f1_scores[fold_id].items():
        print("{} : {:.5f},".format(k,v), end=" ")
    print("")

0:  accuracy : 0.71000, precision : 0.73684, recall : 0.59574, f1-score : 0.65882, 
1:  accuracy : 0.80000, precision : 0.79070, recall : 0.75556, f1-score : 0.77273, 
2:  accuracy : 0.69000, precision : 0.67391, recall : 0.65957, f1-score : 0.66667, 
3:  accuracy : 0.74000, precision : 0.60000, recall : 0.70588, f1-score : 0.64865, 
4:  accuracy : 0.72000, precision : 0.67500, recall : 0.64286, f1-score : 0.65854, 
5:  accuracy : 0.77000, precision : 0.79070, recall : 0.70833, f1-score : 0.74725, 
6:  accuracy : 0.79000, precision : 0.80645, recall : 0.62500, f1-score : 0.70423, 
7:  accuracy : 0.78000, precision : 0.80000, recall : 0.69565, f1-score : 0.74419, 
8:  accuracy : 0.75758, precision : 0.71053, recall : 0.67500, f1-score : 0.69231, 
9:  accuracy : 0.73737, precision : 0.68750, recall : 0.75000, f1-score : 0.71739, 
